In [0]:
%reset

# GLOBAL DECLEARATIONS

In [0]:
import datetime
import os
import numpy as np
from scipy.io import loadmat
from scipy.signal import medfilt
import pywt
from pywt import wavedec
import matplotlib
import matplotlib.pyplot as plt
from PIL import Image
import matplotlib.ticker as ticker
# dbname_mat folders exist here
global_dir = '/content/drive/My Drive/Masters/workdir/ecg_data'
print('GLOBAL DATA DIR :: '+global_dir)

# dbname_npy shoud exsist
global_npydir = os.path.join(global_dir, 'db_npy') #global_dir + 'db_npy/'
print('GLOBAL NPY DIR :: '+global_npydir)

global_datadir = os.path.join(global_npydir, 'data_npy') 
print('GLOBAL DATA DIR :: '+global_datadir)

# global annotation and mapping, common for all ECG from https://physionet.org/about/database/
global_annot = os.path.join(global_dir, 'annotations.txt' )  
print('GLOBAL ANNOTATIONS :: '+global_annot)
# view this file on drive :: https://drive.google.com/open?id=1edhzatX7D3Uo03-d08Lksp807ULlcpgo

#===================================================================================================================
# load MIT annotation mapping  

g_MIT_AMAP_data = np.loadtxt(global_annot, dtype='str',delimiter="\t")
g_MIT_AMAP={}
for a in g_MIT_AMAP_data:
    g_MIT_AMAP[a[0]]= int(a[1])
    #a[0] = orignal mit label (char)
    #a[1] = mapped integer label (int)
    #a[2] = description (str)

map_keys = g_MIT_AMAP.keys()
map_vals = g_MIT_AMAP.values()

print(map_keys)
print(' :: STD LABEL MAPPING :: ')
for k in map_keys:
    print(k+'\t'+str(g_MIT_AMAP[k]))
#print(g_MIT_AMAP)

maxlabels=max(map_vals)+1
print('Max DB Label = '+ str(maxlabels))

g_COLORS = ['green','gold','red']
g_LABELS = np.array(['N','B','A'])
g_COUNT = len(g_LABELS)
print('STD Labels : ' + str(g_LABELS))
print('MAX STD Label = '+ str(g_COUNT))
#===================================================================================================================

print(datetime.datetime.now())


# CLASS DEFINITIONS

In [0]:

#---------------------------------------------------------------------------------------------------------------------------------------------
# CLASS ecg_record : represents one ECG Record in any database
#---------------------------------------------------------------------------------------------------------------------------------------------
class ecg_record:

    def __init__(self, db, recname):
        self.db = db                                # class:{ecg_db}    object this record belongs to
        self.rec = recname                          # string            name of this record
        self.name = db.name + '_'+ recname          # string            full name including db.name

        #self.dirs = db.get_dirs(recname)            # dict              dict of npy data file names to be loaded in data dict used in self.load_data('key')
        self.data = {}                              # dict              dict of npy data file content used in self.read_data('key')

        # find the gain
        info = get_minfo(os.path.join(db.dir_mat,recname+'m.hea'),db.lead2_index+1)  # head has this info at lead2's index
        self.gain = int(info[2].split('/')[0])
        #print(self.info)
        
    def read_data(self, data_type):
        return self.data[data_type] #= np.load(self.dirs[s])

    def load_data(self, data_type):
        self.data[data_type] = np.load(os.path.join(self.db.dir_npy, self.rec + '_'+data_type+'.npy'))
        return self.data[data_type]

#---------------------------------------------------------------------------------------------------------------------------------------------


#---------------------------------------------------------------------------------------------------------------------------------------------
# CLASS ecg_db : represents one ECG database
#---------------------------------------------------------------------------------------------------------------------------------------------
class ecg_db:
    def __init__(self, dbname,  sampling_rate, lead2_index):
        print('\nInitailze new ecg database ... ')
        self.name = dbname  #str
        self.srate = sampling_rate #int
        self.lead2_index = lead2_index #int
        #self.dir_raw = global_dir + dbname+'_raw/' #str
        self.dir_mat = os.path.join(global_dir , dbname+'_mat') #str
        self.dir_npy = os.path.join(global_npydir , dbname+'_npy') #str
        self.recs = set(np.loadtxt(os.path.join(self.dir_mat,'RECORDS'), dtype='str',delimiter="\n")) #set
        self.info()

    def info(self):
        print( 'DB NAME :: '+ self.name)
        print( 'MAT DIR :: ' + self.dir_mat )
        print( 'NPY DIR :: ' + self.dir_npy )
        print( 'S_RATE :: ' + str(self.srate))
        print( 'LEAD2 INDEX  :: ' + str(self.lead2_index))
        print( 'RECORD SET :: [' +str(len(self.recs))+'] ' + str(self.recs) )
        return 0

    def get_record(self,rec):
        return ecg_record(self,rec)

#---------------------------------------------------------------------------------------------------------------------------------------------


#---------------------------------------------------------------------------------------------------------------------------------------------
# Pre processing Functions
#---------------------------------------------------------------------------------------------------------------------------------------------

def denoise_signal(X, dwt_transform, dlevels, cutoff_low, cutoff_high):
    coeffs = wavedec(X, dwt_transform, level=dlevels)   # wavelet transform 'bior4.4'
    # scale 0 to cutoff_low 
    for ca in range(0,cutoff_low):
        coeffs[ca]=np.multiply(coeffs[ca],[0.0])
    # scale cutoff_high to end
    for ca in range(cutoff_high, len(coeffs)):
        coeffs[ca]=np.multiply(coeffs[ca],[0.0])
    Y = pywt.waverec(coeffs, dwt_transform) # inverse wavelet transform
    return Y  

def get_median_filter_width(sampling_rate, duration):
    res = int( sampling_rate*duration )
    if res % 2 == 0 : # is even
        res+=1 # make odd
    return res

def filter_signal(X,mfltrs_array):
    X0 = X  #read orignal signal
    for mi in range(0,len(mfltrs_array)):
        X0 = medfilt(X0,mfltrs_array[mi]) # apply median filter one by one on top of each other
    X0 = np.subtract(X,X0)  # finally subtract from orignal signal
    return X0

def get_minfo(minfo_path, minfo_line_index):
    f = open(minfo_path,'r')
    ls = f.readlines()
    f.close()
    la = ls[minfo_line_index].split()    #I67m.mat 16+192 623/mV 16 0 4441 1894 0 II
    return la # gain at 2 baseline at 4

#---------------------------------------------------------------------------------------------------------------------------------------------


#---------------------------------------------------------------------------------------------------------------------------------------------
# Database Building Functions
#---------------------------------------------------------------------------------------------------------------------------------------------

#---------------------------------------------------------------------------------------------------------------------------------------------
# 1 Build Annotations :: a.txt -> ant.npy
def build_db_ants(self_db, output_postfix):  
#---------------------------------------------------------------------------------------------------------------------------------------------
    xs=len(self_db.recs)
    #xc=1
    print('\nBEGIN TASK :: Build annotations for ' + str(xs)+ ' records')
    print('REC\tFound\tKnown\tUnknown' )
    for rec in self_db.recs:
        #print('REC: '+rec+ '...'+str(xc)+ ' of '+ str(xs))
        #xc+=1
        ants = np.loadtxt(os.path.join(self_db.dir_mat , rec + 'a.txt'), dtype='str', delimiter= '\n', skiprows = 1)
        #print('\tFound Annotations\t'+str(len(ants)))
        antsnpy = np.zeros((0,2),dtype='str')
        #print('  annotations found '+str(len(ants)))
        for i in range(0,len(ants)):
            a_a = ants[i].split()
            a_sample = a_a[1]
            a_label = a_a[2]
            if (a_label in map_keys):
                antsnpy = np.vstack((antsnpy,np.array([a_sample,a_label]))) # sample,label str
            else:
                print('\tWARNING :: Unknown annotation "'+a_label+'" found at [' +a_sample+ '] in file a.txt, skipping....')
        
        adf =  len(ants) - len(antsnpy) 
        #print('\tKnown Annotations\t'+str(len(antsnpy)))
        # print('\tUnknown Annotations\t'+str(adf))
        print(rec + '\t'+ str(len(ants)) + '\t'+str(len(antsnpy))+'\t' + str(adf) )
        # save for this record
        np.save(os.path.join(self_db.dir_npy , rec+ '_'+output_postfix+'.npy'), antsnpy)
        #print('\tSaved at "'+save_loc+'"')
    print('\nEND TASK :: Build annotations for ' + str(xs)+ ' records')
    return 0
#---------------------------------------------------------------------------------------------------------------------------------------------


#---------------------------------------------------------------------------------------------------------------------------------------------
# 2 Map Annotations :: ant.npy -> antm.npy
def map_db_ants(self_db, g_map, input_postfix, output_postfix):
#---------------------------------------------------------------------------------------------------------------------------------------------
    #global g_MIT_AMAP
    xs=len(self_db.recs)
    #xc=1
    print('\nTASK :: MAP annotations for ' + str(xs)+ ' records :: using prefix '+ '*_'+output_postfix+'.npy')
    print('REC\tFound\tMapped\tUnmapped' )
    for rec in self_db.recs:
        #print('REC: '+rec+ '...'+str(xc)+ ' of '+ str(xs))
        #xc+=1

        ants = np.load(os.path.join( self_db.dir_npy , rec + '_'+input_postfix+'.npy'))

        #print('\tFound Annotations\t'+str(len(ants)))
        antsmap = np.zeros((0,2),dtype='int')
        #print('  annotations found '+str(len(ants)))
        for i in range(0,len(ants)):
            a_sample = int(ants[i][0])
            a_label = g_map[ants[i][1]]
            if a_label>=0:
                antsmap = np.vstack((antsmap, np.array([a_sample,a_label])))

        adf =  len(ants) - len(antsmap) 
        #print('\tMapped Annotations\t'+str(len(antsmap)))
        #print('\tUnmapped Annotations\t'+str(adf))
        print(rec + '\t'+ str(len(ants)) + '\t'+str(len(antsmap))+'\t' + str(adf) )
        # save for this record
        np.save(os.path.join(  self_db.dir_npy , rec + '_'+output_postfix+'.npy'), antsmap)
        #print('\tSaved at "'+save_loc+'"')
    print('\nEND TASK :: MAP annotations for ' + str(xs)+ ' records')
    return 0
#---------------------------------------------------------------------------------------------------------------------------------------------


#---------------------------------------------------------------------------------------------------------------------------------------------
# 3 Extract Raw lead2 signal :: m.mat[lead2] -> raw2.npy
def build_db_signal(self_db, output_postfix):  # saves xm.mat[lead2] --> x_raw2.npy
#---------------------------------------------------------------------------------------------------------------------------------------------
    xs=len(self_db.recs)
    xc=0
    print('\nBEGIN TASK :: Save lead2 signal data for ' + str(xs)+ ' records')
    for rec in self_db.recs:
        xc+=1
        print('Rec: '+rec+ ' ... '+ str(xc)+' of '+ str(xs))
        mat_signal = loadmat(os.path.join( self_db.dir_mat , rec + 'm.mat'))['val'][self_db.lead2_index]
        np.save(os.path.join( self_db.dir_npy , rec + '_'+output_postfix+'.npy'), mat_signal)    
    print('END TASK :: Save lead2 signal data for ' + str(xs)+ ' records')
    return 0
#---------------------------------------------------------------------------------------------------------------------------------------------


#---------------------------------------------------------------------------------------------------------------------------------------------
# 4 Filter Raw/Denoised signal :: raw2.npy -> med2.npy or den2.npy -> rmd2.npy
def filter_db_signal(self_db, ms_flt_array, input_postfix, output_postfix): 
#---------------------------------------------------------------------------------------------------------------------------------------------
    xs=len(self_db.recs)
    xc=0
    mfa = np.zeros(len(ms_flt_array), dtype='int')
    #print('fun')
    for i in range(0, len(ms_flt_array)):
        mfa[i] = get_median_filter_width(self_db.srate,ms_flt_array[i])
        #print('loop')
    print('\nBEGIN TASK :: filer lead2 signal data for ' + str(xs)+ ' records:' + input_postfix + ' --> ' + output_postfix)
    for rec in self_db.recs:
        xc+=1
        print('Rec: '+rec+ ' ... '+ str(xc)+' of '+ str(xs))
        X = np.load(os.path.join( self_db.dir_npy , rec + '_'+input_postfix+'.npy')) #np.save(rec_fnpy + '_raw2.npy', mat_signal) 
        X_med2 = filter_signal(X, mfa)
        np.save(os.path.join( self_db.dir_npy , rec + '_'+output_postfix+'.npy'),X_med2) # save a copy of (median filtered ) signal = r,m  
        #----------------------------------------------------------------------         
    print('END TASK :: filer lead2 signal data for ' + str(xs)+ ' records')
    return 0
#---------------------------------------------------------------------------------------------------------------------------------------------


#---------------------------------------------------------------------------------------------------------------------------------------------
# 5 Denoise Raw/Filtered signal :: med2.npy -> rdm2.npy or raw2.npy -> den2.npy
def denoise_db_signal(self_db, dwt_transform, dlevels, cutoff_low, cutoff_high, input_postfix, output_postfix):
#---------------------------------------------------------------------------------------------------------------------------------------------
    xs=len(self_db.recs)
    xc=0
    
    print('\nBEGIN TASK :: denoising lead2 signal data for ' + str(xs)+ ' records:' + input_postfix + ' --> ' + output_postfix)
    for rec in self_db.recs:
        xc+=1
        print('Rec: '+rec+ ' ... '+ str(xc)+' of '+ str(xs))
        X = np.load(os.path.join( self_db.dir_npy , rec + '_'+input_postfix+'.npy') ) # np.load(self_db.dir_npy + rec + '_raw2.npy')
        X_rlt2 = denoise_signal(X,dwt_transform, dlevels, cutoff_low, cutoff_high )
        np.save(os.path.join( self_db.dir_npy , rec + '_'+output_postfix+'.npy'),X_rlt2) # save a copy of (only denoised)  signal = r,m,d 
        #----------------------------------------------------------------------         
    print('END TASK :: denoising lead2 signal data for ' + str(xs)+ ' records')
    return 0
#---------------------------------------------------------------------------------------------------------------------------------------------
#---------------------------------------------------------------------------------------------------------------------------------------------


# DB DICT CREATION

In [0]:
# def __init__(self, dbname,  sampling_rate, lead2_index):
all_db = {}
all_db['mitdb'] = ecg_db('mitdb', 360, 0)
all_db['svdb'] = ecg_db('svdb',  128, 0)
all_db['incartdb'] = ecg_db('incartdb', 257, 1)
#print(all_db.values())
print('')
print(all_db.keys())


g_MIT_AMAP_POSTFIX = 'ant_m'
g_ANT_POSTFIX = 'ant'
g_RAW_POSTFIX = 'raw2'
g_MED_POSTFIX = 'med2'
g_DEN_POSTFIX = 'den2'
g_RMD_POSTFIX = 'rmd2'
g_RDM_POSTFIX = 'rdm2'


x_scale = 25 * 0.04  # 25 mm or 1 inches  per second
y_scale = 10 * 0.04  # 10 mm or 0.4 inches  per mV
m_scale = 2          # multiplier (zoom)
fylims = (-2.5,2.5) #mV
fhig = (fylims[1]-fylims[0]) * y_scale * m_scale

fdpi=150



# [] E X T R A C T I N G  *  D A T A B A S E *

 C A R E F U L   
 T O    
 R U N    
 T H I S   
 C E L L




In [0]:
#    C A R E F U L   D O N T     R U N    T H I S 




db_data = all_db.values()
# input output postfix = _[postfix].npy
print(datetime.datetime.now())
print('S T A R T \n')

for db in db_data:
#=========================================================================================
    print ('\n START ON DATABASE: '+ db.name +' \t'+ str(datetime.datetime.now()))
    
    #try: 
    #    directory = os.makedirs(os.path.join(global_npydir, db.name+'_npy') , exist_ok = True) 
    #    print("Directory '%s' created successfully" % directory) 
    #except OSError as error: 
     #   print("Directory '%s' can not be created" % directory) 
#=========================================================================================

    print ('\n START COPY ANNOTATIONS \t'+ str(datetime.datetime.now()))
    build_db_ants(db, g_ANT_POSTFIX) # build_db_ants(self, output_postfix)

#=========================================================================================

    print ('\n START MAP ANNOTATIONS \t'+ str(datetime.datetime.now()))
    map_db_ants(db, g_MIT_AMAP,g_ANT_POSTFIX,g_MIT_AMAP_POSTFIX) #map_db_ants(self, g_map, input_postfix, output_postfix)

#=========================================================================================

    print ('\n START EXTRACTING RAW SIGNAL [ R ] \t'+ str(datetime.datetime.now()))
    build_db_signal(db, g_RAW_POSTFIX) # build_db_signal(self, output_postfix)

#=========================================================================================

    print ('\n START FILTERING RAW SIGNAL [ R M ] \t'+ str(datetime.datetime.now()))
    #print('this')
    filter_db_signal(db, [0.2,0.6],g_RAW_POSTFIX,g_MED_POSTFIX) # filter_db_signal(self, ms_flt_array, input_postfix, output_postfix)
    print ('\n START DENOISE FILTERED SIGNAL [ R M D ] \t'+ str(datetime.datetime.now()))
    denoise_db_signal(db, 'bior4.4', 11 , 2 , 9, g_MED_POSTFIX, g_RMD_POSTFIX ) # denoise_db_signal(self, dwt_transform, dlevels, cutoff_low, cutoff_high, input_postfix, output_postfix):

#=========================================================================================

    print ('\n START DENOISE RAW SIGNAL [ R D ]\t'+ str(datetime.datetime.now()))
    denoise_db_signal(db, 'bior4.4', 11 , 2 , 9, g_RAW_POSTFIX, g_DEN_POSTFIX ) # denoise_db_signal(self, dwt_transform, dlevels, cutoff_low, cutoff_high, input_postfix, output_postfix):
    print ('\n START FILTERING DENOISE SIGNAL [ R D M ] \t'+ str(datetime.datetime.now()))
    filter_db_signal(db, [0.2,0.6],g_DEN_POSTFIX, g_RDM_POSTFIX) # filter_db_signal(self, ms_flt_array, input_postfix, output_postfix)
#=========================================================================================
    print ('\n END ON DATABASE:'+ db.name +' \t'+ str(datetime.datetime.now()))
#=========================================================================================
print('\n E N D ')
print(datetime.datetime.now())
#    C A R E F U L   D O N T     R U N    T H I S 

# VIEW Section

In [0]:
# Stated at 04:38
#db_data = [mit_db,sv_db,inc_db]
#dbs = all_db['mitdb']
#print(dbs.recs)

#selrec = dbs.get_record('233')
#manual_gain=1/selrec.gain
#print('gain='+str(selrec.gain))

idbs = 'incartdb'  # 'mitdb'  'svdb' 'incartdb'
irec = 'I48'

# VIEW 1 :: RECORD

In [0]:
dbs = all_db[idbs]
selrec = dbs.get_record(irec)
manual_gain=1/selrec.gain


#dload = ['ant','antm','raw2','med2','flt2','rlt2']
dload = [g_ANT_POSTFIX,g_MIT_AMAP_POSTFIX]#, g_MED_POSTFIX]
for dt in dload:
    selrec.load_data(dt)

ant2 = selrec.read_data(g_ANT_POSTFIX)
ant2m = selrec.read_data(g_MIT_AMAP_POSTFIX) # only has mapr>=0 {no umapped beats preset}
#sig2 = selrec.read_data(g_MED_POSTFIX)
#print(selrec.name)
#print('Total STD labels = '+ str(len(ant2m)))
ant_s = ant2[:,0].astype('int')
ant_l = ant2[:,1]

ant_sm = ant2m[:,0]
ant_lm = ant2m[:,1]

#

samp_dict ={}
plt.figure(5, figsize=(240,4),dpi=100)
plt.ylim(-0.2*g_COUNT-1, 1.5 )
plt.xlim(-10, 1830 )

aa = plt.axes()
ax = aa.get_xaxis()
ay = aa.get_yaxis()

ax.set_major_locator(ticker.MultipleLocator(5))
ay.set_visible(False)

#plt.ylabel('class label')
#plt.xlabel('time (sec)')
plt.grid(axis='x' )

#density in 10 ms window
den_N = np.zeros((1800,3),dtype=float)


for i in range(0, len(den_N)):
    temp = ant2m[(ant_sm>(i-5)*selrec.db.srate) & (ant_sm<(i+5)*selrec.db.srate)] [:,1]
    
    den_N[i,0]=len(temp[temp==0]) / len(temp) 
    den_N[i,1]=len(temp[temp==1]) / len(temp)  
    den_N[i,2]=len(temp[temp==2]) / len(temp)  


for gi in range(0,g_COUNT):
    samp_dict = ant_sm[ant_lm==gi]
    plt.scatter(np.multiply(samp_dict,[1/dbs.srate]),np.subtract(np.multiply(ant_lm[ant_lm==gi],[-0.2]),1),color=g_COLORS[gi],marker='.',label=g_LABELS[gi]+' ['+str(len(samp_dict))+']')


#------------------------------ CUSTOM HERE
cust_label = 'Q'
ps_x = np.multiply(ant_s[ant_l==cust_label],[1/dbs.srate])
ps_y = np.subtract(np.zeros(len(ant_l[ant_l==cust_label])),[0.75])
plt.scatter(ps_x,ps_y,color='blue',marker='o',label='Custom: ' + cust_label+' ['+str(len(ps_x))+']')
#print('"'+cust_label+'"\t'+str(len(ps_x)))
#------------------------------ CUSTOM HERE

#print('----------------------------------')

ps_x = np.multiply(ant_s[ant_l=='x'],[1/dbs.srate])
ps_y = np.subtract(np.zeros(len(ant_l[ant_l=='x'])),[0.25])
plt.scatter(ps_x,ps_y,color='black',marker='x',label='Non-Cond P-wave'+' ['+str(len(ps_x))+']')
#print('Non-Cond P-wave :: '+str(len(ps_x)))

ps_x = np.multiply(ant_s[ant_l=='|'],[1/dbs.srate])
ps_y = np.subtract(np.zeros(len(ant_l[ant_l=='|'])),[0.25])
plt.scatter(ps_x,ps_y,color='black',marker='|',label='QRS-like complex'+' ['+str(len(ps_x))+']')
#print('QRS-like complex :: '+str(len(ps_x)))


ps_x = np.multiply(ant_s[ant_l=='~'],[1/dbs.srate])
ps_y = np.subtract(np.zeros(len(ant_l[ant_l=='~'])),[0.5])
plt.scatter(ps_x,ps_y,color='red',marker='x',label='~Quality change~'+' ['+str(len(ps_x))+']')
#print('~Quality change~ :: '+str(len(ps_x)))

ps_x = np.multiply(ant_s[ant_l=='"'],[1/dbs.srate])
ps_y = np.subtract(np.zeros(len(ant_l[ant_l=='"'])),[0.5])
plt.scatter(ps_x,ps_y,color='black',marker='+',label='Commnet'+' ['+str(len(ps_x))+']')
#print('Commnet :: '+str(len(ps_x)))

ps_x = np.multiply(ant_s[ant_l=='+'],[1/dbs.srate])
ps_y = np.subtract(np.zeros(len(ant_l[ant_l=='+'])),[0.5])
plt.scatter(ps_x,ps_y,color='green',marker='>',label='Rhythm change'+' ['+str(len(ps_x))+']')
#print('Rhythm change :: '+str(len(ps_x)))

#print('----------------------------------')
plt.plot(den_N[:,0],color= 'green',linewidth=0.8)
plt.plot(den_N[:,1],color= 'gold',linewidth=0.8)
plt.plot(den_N[:,2],color= 'red',linewidth=0.8)
#plt.plot(avg_N, 'b-',)

#plt.plot(np.multiply(sig2,[1/selrec.gain]),linewidth=0.2, color='black')
plt.title(selrec.name + ':: Total peaks = '+str(len(ant2m)))
plt.legend()

#plt.yticks(np.arange(0,1800,1))
    #print(selsamp)
# nos output = div (L-d)/s

# find continuous range
#print(ant2[ant_l=='x'])


# VIEW 2 :: VIEW SAVE

In [0]:
#============ select interval start times
time_safe = 3
time_period = 10 #seconds
# choose random start times in range 10,1790
# 1 FORM Abnormal beats databse
# from each record take at least 
time_start = np.array( 
                [10,20,45,107,127,150, 190,255, 420,440,535,660, 795,845, 1100,1320,1455,1700, 1770]
                )  #900 #seconds
# mitdb_232 = [20,45,105,135,165,190,440,505,575,635,795,890,943,1070,1270,1395,1435,1520,1580,1635,1680,1710,1760,1785]
# mitdb_208 = [75,105,130,185,235,285,430,495,545,585,625,665,755,815,980,1005,1035,1140,1305,1465,1590,1675,1700,1735]
# mitdb_200 = [75,105,130,185,235,285,430,495,545,585,625,665,755,815,980,1005,1035,1140,1305,1465,1590,1675,1700,1735]
#------------------------
#g_RAW_POSTFIX = 'raw2'
#g_MED_POSTFIX = 'med2'
#g_DEN_POSTFIX = 'den2'
#g_RMD_POSTFIX = 'rmd2'
#g_RDM_POSTFIX = 'rdm2'
#dload = [g_RAW_POSTFIX,g_MED_POSTFIX,g_DEN_POSTFIX,g_RMD_POSTFIX,g_RDM_POSTFIX]
#for dt in dload:
#    selrec.load_data(dt)

#signalraw2 = selrec.load_data(g_RAW_POSTFIX)

signalmed2 = selrec.load_data(g_MED_POSTFIX)
#signalden2 = selrec.load_data(g_DEN_POSTFIX)
signalrmd2 = selrec.load_data(g_RMD_POSTFIX)
#signalrdm2 = selrec.load_data(g_RDM_POSTFIX)
#med2 = selrec.read_data(g_MED_POSTFIX)    # med2 = selrec.read_data('med2')


L = len(signalmed2)

#time_start = np.array([285,635,1405,1660])  #900 #seconds



#-------------------------------------------------------------------------------------------------------
desc = ''
desc += 'REC\t'+ str(selrec.name) + '\n'
#desc += 'Time\t'+ str(time_start) + ' : ' +str(time_start+time_period) +'\n'
desc += 'Params\tX='+ str(x_scale) + ' in/sec, Y='+str(y_scale)+' in/mV'+', Multiplier=' +str(m_scale)+ '\n'
desc += 'Params\tY-RANGE='+str(fylims)+' mV, Manual_Gain='+str(manual_gain)+'\n'
#-------------------------------------------------------------------------------------------------------
print(desc)

time_safe *= dbs.srate
time_period *= dbs.srate
time_start = np.multiply(time_start,[dbs.srate])
allfig = 0
#for_saving= []
for tstart in time_start:

    fi = (tstart)
    ti = (tstart+time_period)
    save_prefix = selrec.name+'_'+str(fi)+'_'+str(ti)       # name = dbname_rec_fi_ti
    #print( 'DUR\t'+ str(round(time_period/dbs.srate,2)) + 'sec ['+ str(round(tstart/dbs.srate,2)) + ' : ' +str(round((tstart+time_period)/dbs.srate,2)) +']' )
    #print(str(fi)+':'+str(ti)+ ' | '+ str(L))
    if fi<0 or ti>L:
        print ('Beyond Limits : can not form this view')
        continue

    #fsignalraw2 = np.multiply(signalraw2[fi:ti],[manual_gain])
    fsignalmed2 = np.multiply(signalmed2[fi:ti],[manual_gain])
    #fsignalden2 = np.multiply(signalden2[fi:ti],[manual_gain])
    fsignalrmd2 = np.multiply(signalrmd2[fi:ti],[manual_gain])
    #fsignalrdm2 = np.multiply(signalrdm2[fi:ti],[manual_gain])

     #----------visual figure------------------------------------------------------------------------------------------------
    # determine size of figure
    fwid = ((ti-fi)/dbs.srate) * x_scale * m_scale
    lysize = (fwid,fhig*1) #secods

    plt.figure(allfig,figsize=lysize, frameon=False, dpi=100)
    allfig+=1

    plt.ylim(fylims)
    plt.xlim((0,ti-fi))
    plt.axes().get_yaxis().set_visible(False)
    plt.grid(axis='x')

    # plot signals
    #plt.plot(fsignalraw2,linewidth=0.2,color='black', label = 'raw2')
    #plt.plot(fsignal1,linewidth=0.2,color='green', label = 'med2')
    #plt.plot(fsignal2,linewidth=0.5,color='gold', label = 'den2')
    plt.plot(fsignalmed2,linewidth=0.2,color='black', label = 'med')
    plt.plot(fsignalrmd2,linewidth=0.2,color='blue', label = 'rmd2')
    #plt.plot(np.subtract(fsignalmed2,fsignalrmd2),linewidth=0.2,color='gold', label = 'rm-rmd')
    #plt.plot(np.subtract(fsignal4,fsignal3),linewidth=0.2,color='gold', label = 'diff rdm2 - rmd2')

    # plot annotations as ticks
    ar_s = np.subtract(ant_s[(ant_s>=fi) & (ant_s<ti)], fi)#[1:-1] # skip first and last R peak
    ar_l = ant_l[(ant_s>=fi) & (ant_s<ti)]#[1:-1] # skip first and last R peak
    plt.xticks(ar_s,ar_l)
    plt.title(str(allfig)+' '+save_prefix)

    #plt.legend()
    
    #----------visual figure------------------------------------------------------------------------------------------------


    #----------data figure------------------------------------------------------------------------------------------------

    # select which signal to save


    # select annotations to save as labels====================================
    ar_sm = np.subtract(ant_sm[(ant_sm>=fi) & (ant_sm<ti)], fi)
    ar_lm = ant_lm[(ant_sm>=fi) & (ant_sm<ti)]

    #time_safe = 3*dbs.srate # 
    if ar_sm[1] < time_safe: # 1st beat is below 3 sec mark -> skip it
        ar_sm = ar_sm[1:] # skip first and last R peak
        ar_lm = ar_lm[1:]

    if ar_sm[-1] > time_period - time_safe:
        ar_sm = ar_sm[:-1] # skip first and last R peak
        ar_lm = ar_lm[:-1]

    
    vinfo = np.transpose(np.vstack((ar_sm,ar_lm)))
    
    print(vinfo)
    print(vinfo.shape)
    print(len(vinfo))
    #for_saving.append(vinfo)
    #np.save(os.path.join(global_datadir,save_prefix+'_vi.npy'), fs)
    np.save(os.path.join(global_datadir,save_prefix+'_vi.npy'), vinfo)
    #plt.savefig(os.path.join(global_datadir,save_prefix+'_vinfo.png'))


    # continuous labels
    #labels_yc = np.subtract(np.zeros(ti-fi,dtype='int'),[1],dtype='int') # -1 by default, >=0 for label in g_LABELS
    #for j in range(0,len(ar_sm)):# should skip first and last
    #    labels_yc[ ar_sm[j] ] = ar_lm[j]
    # the label is same as the integer label mapped by AMAP >=0, so minimum is always 0 rest are -1


# save it
    #data_x = fsignalmed2
    

    #np.save(os.path.join(global_datadir,save_prefix+'_xr2.npy'),fsignalraw2)
    #np.save(os.path.join(global_datadir,save_prefix+'_xrm2.npy'),fsignalmed2)
    #np.save(os.path.join(global_datadir,save_prefix+'_xrd2.npy'),fsignalden2)
    #np.save(os.path.join(global_datadir,save_prefix+'_xrmd2.npy'),fsignalrmd2)
    #np.save(os.path.join(global_datadir,save_prefix+'_xrdm2.npy'),fsignalrdm2)

    #np.save(os.path.join(global_datadir,save_prefix+'_y.npy'), labels_y)
    #np.save(os.path.join(global_datadir,save_prefix+'_yc.npy'),labels_yc)

    
    #plt.close()
    # select annotations to save as labels====================================


    # build info dict s====================================
#    samp_dict1 ={} # this is local dictionary ... needed
#    for gi in range(0,g_COUNT):
#        samp_dict1[gi] = ar_sm[ar_lm==gi]
#        print(str(g_LABELS[gi])+'\t'+str(len(samp_dict1[gi])))
        
    #plt.figure(1, figsize=fysize, frameon=False, dpi=fdpi)
    #plt.ylim(fylims)
    #plt.xlim((0,ti-fi))
    #plt.axes().get_yaxis().set_visible(False)
    #plt.axis('off')
    ##plt.grid('both')
    #plt.plot(fsignal1,linewidth=0.2,color='gold')       #<------------------------------ which signal is is the final data signal ?? [raw,med,flt,rlt]
    ##plt.xticks(ar_sm,[],color='grey')

    #plt.figure(0,figsize=fysize, frameon=False, dpi=fdpi)    # SAVE PLOT AND LABELS====================================================================================
    #plt.ylim((-1.5,maxlabels+1.5))
    #plt.xlim((0,ti-fi))

    #plt.axes().get_yaxis().set_visible(False)
    #plt.axis('off')
    #plt.grid('both')

    #plt.plot(labels_y, linewidth=0.4, color='black') # SAVE THIS LABEL DATA
    #plt.scatter(labels_y1[0],labels_y1[1],marker='o', color='black')
    #for i in range(0,g_COUNT):
    #    plt.scatter(ar_sm[ar_lm==i],ar_lm[ar_lm==i], color=g_COLORS[i], label=g_LABELS[i],marker='.')
        
    ##plt.xticks(ar_sm,[])
    #plt.legend()





In [0]:
flist = os.listdir(global_datadir)
for i in range(0,len(flist)):
    print(str(i)+'\t'+ str(flist[i]))

In [0]:

fli = flist[9]
args = fli.split('_')
print(args)
dbi = args[0]
reci = args[1]
fi = int(args[2])
ti = int(args[3])


save_prefix = dbi+'_'+reci+'_'+str(fi)+'_'+str(ti)    # name = dbname_rec_fi_ti

vinfo = np.load(os.path.join(global_datadir,fli))


my_db = all_db[dbi]
my_rec = my_db.get_record(reci)
manual_gain=1/my_rec.gain
#print('gain='+str(my_rec.gain))
signalmed2 = np.multiply(my_rec.load_data(g_MED_POSTFIX)[fi:ti],[manual_gain])

fwid = ((ti-fi)/my_db.srate) * x_scale * m_scale
lysize = (fwid,fhig*1) #secods

plt.figure(0,figsize=lysize, frameon=False, dpi=fdpi)

plt.ylim(fylims)
plt.xlim((0,ti-fi))
plt.axes().get_yaxis().set_visible(False)
plt.grid(axis='x')

plt.plot(signalmed2,color='blue',linewidth=0.2)
plt.xticks(vinfo[:,0],vinfo[:,1])
plt.title(save_prefix)
